In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt
import math
import statistics
from statistics import mean

In [2]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost.sklearn import XGBRegressor

# Pre-processing

## Training set  (Pre-processed)

In [3]:
filename='./Data/df_train1.csv'
df_train1 = pd.read_csv(filename)

In [4]:
df_train1

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,1665
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,472
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,1616
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,2434
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,2043
...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,339
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,1149
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,817
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,477


### Read One-hot matrix from csv - process see 1118_genres&categories&tags.ipynb

In [5]:
filename='./Data/train_One_hot_genres.csv'
dfg_train = pd.read_csv(filename)

In [6]:
df_train2 = pd.concat([df_train1, dfg_train], axis=1, sort=False)
df_train2

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,...,0,0,0,0,0,0,0,0,0,0
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,...,0,0,1,0,0,0,0,0,0,0
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,...,0,0,0,0,0,1,0,0,0,0
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,...,0,0,0,0,0,0,0,0,0,0
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,...,0,0,1,0,0,0,0,0,0,0
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,...,0,0,0,0,0,0,0,0,0,0


## Testing set  (Pre-processed)

In [7]:
filename='./Data/df_test1.csv'
df_test = pd.read_csv(filename)

In [8]:
df_test

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,2350.0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,452.0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,848.0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,3850.0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,1282.0
...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,1766.0
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,3092.0
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,1650.0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,153.0


### Read One-hot Matrix from csv - process see 1118_genres&categories&tags.ipynb

In [9]:
filename='./Data/test_One_hot_genres.csv'
dfg_test = pd.read_csv(filename)

In [10]:
df_test2 = pd.concat([df_test, dfg_test], axis=1, sort=False)
df_test2

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,...,0,0,0,0,0,0,0,0,0,0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,...,0,0,0,0,0,0,0,1,0,0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,...,0,0,0,0,0,1,0,1,0,0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,...,0,0,0,0,0,0,0,1,0,0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,...,0,0,0,0,0,0,0,0,0,0
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,...,0,0,0,0,0,0,0,0,0,0
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,...,0,0,0,0,0,1,0,1,0,0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,...,0,0,1,0,0,1,1,1,0,0


# Feature selection

In [11]:
for d in zip(df_train2, df_test2):
    if d[0] != d[1]:
        print (d)

('playtime_forever', 'id')


In [12]:
scaler = StandardScaler()

X = df_train2.drop(columns=['playtime_forever', 'is_free', 'genres', 'categories', 'tags', 'purchase_date','release_date'])
test_X = df_test2.drop(columns=['id', 'is_free', 'genres', 'categories', 'tags', 'purchase_date','release_date'])
y= df_train2.playtime_forever

X_scaled = scaler.fit(X).transform(X)
y_log = np.log(df_train2.playtime_forever + 1)
test_X_scaled = scaler.transform(test_X)

In [13]:
X_scaled

array([[-0.07482934, -0.35131016, -0.21882637, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.07874784, -0.36050227, -0.23019318, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.07345257, -0.28161873, -0.15169116, ..., -0.54562801,
        -0.05431254, -0.13403012],
       ...,
       [-0.06995769, -0.2268085 , -0.02665628, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.07154627, -0.34219707, -0.21136691, ..., -0.54562801,
        -0.05431254, -0.13403012],
       [-0.0681573 , -0.3370084 , -0.21811595, ...,  1.83275049,
        -0.05431254, -0.13403012]])

# Modeling & Evaluation 

In [14]:
# define cross validation stragegy RMSE
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [15]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(),SGDRegressor(),BayesianRidge(),KernelRidge(),
          ExtraTreesRegressor(),XGBRegressor(verbosity=0),AdaBoostRegressor()]

In [16]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb","Ada"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_scaled, y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 13355495097098.308594, 26710990194194.8203
Ridge: 0.865655, 0.1264
Lasso: 0.883973, 0.1248
RF: 0.819736, 0.1071
GBR: 0.872681, 0.1225
SVR: 0.844382, 0.1326
LinSVR: 0.967992, 0.1477
Ela: 0.883973, 0.1248
SGD: 1.004095, 0.2510
Bay: 0.825339, 0.1228
Ker: 6.026479, 4.5220
Extra: 0.839768, 0.1167
Xgb: 0.836758, 0.1187
Ada: 0.847998, 0.0851


## Fine Tuning Parameters

In [17]:
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

In [25]:
# Ridge - USABLE

grid(Ridge()).grid_get(X_scaled,y_log,{'alpha':[155,156,157,158,159,160,161,162,163,164,165]})

{'alpha': 161} 0.8295940540343514
            params  mean_test_score  std_test_score
0   {'alpha': 155}         0.829607        0.211478
1   {'alpha': 156}         0.829603        0.211528
2   {'alpha': 157}         0.829600        0.211578
3   {'alpha': 158}         0.829597        0.211628
4   {'alpha': 159}         0.829595        0.211677
5   {'alpha': 160}         0.829594        0.211726
6   {'alpha': 161}         0.829594        0.211775
7   {'alpha': 162}         0.829594        0.211824
8   {'alpha': 163}         0.829596        0.211872
9   {'alpha': 164}         0.829597        0.211920
10  {'alpha': 165}         0.829600        0.211968


In [19]:
# RF - USABLE

grid(RandomForestRegressor()).grid_get(X_scaled,y_log,{'max_features': ["auto","sqrt","log2"]})

{'max_features': 'log2'} 0.8104625069090567
                     params  mean_test_score  std_test_score
0  {'max_features': 'auto'}         0.836182        0.214236
1  {'max_features': 'sqrt'}         0.829086        0.231976
2  {'max_features': 'log2'}         0.810463        0.243350


In [149]:
# Lasso

grid(Lasso()).grid_get(X_scaled,y_log,{'alpha': [0.001,0.005,0.01,0.015,0.02,0.05,0.1],'max_iter':[10000]})

{'alpha': 0.015, 'max_iter': 10000} 0.8487055707444151
                                params  mean_test_score  std_test_score
0  {'alpha': 0.001, 'max_iter': 10000}         0.869101        0.224891
1  {'alpha': 0.005, 'max_iter': 10000}         0.856353        0.218249
2   {'alpha': 0.01, 'max_iter': 10000}         0.849912        0.218573
3  {'alpha': 0.015, 'max_iter': 10000}         0.848706        0.218463
4   {'alpha': 0.02, 'max_iter': 10000}         0.848790        0.220182
5   {'alpha': 0.05, 'max_iter': 10000}         0.857456        0.237627
6    {'alpha': 0.1, 'max_iter': 10000}         0.867640        0.246853


In [168]:
# GBR

grid(GradientBoostingRegressor()).grid_get(X_scaled,y_log,{'learning_rate':[0.011,0.012,0.013,0.014,0.015,0.016,0.018,0.018,0.019]})

{'learning_rate': 0.018} 0.7936447313855081
                     params  mean_test_score  std_test_score
0  {'learning_rate': 0.011}         0.799255        0.221078
1  {'learning_rate': 0.012}         0.797947        0.222645
2  {'learning_rate': 0.013}         0.797853        0.223315
3  {'learning_rate': 0.014}         0.796575        0.221349
4  {'learning_rate': 0.015}         0.793774        0.218609
5  {'learning_rate': 0.016}         0.795310        0.221214
6  {'learning_rate': 0.018}         0.794670        0.224907
7  {'learning_rate': 0.018}         0.793645        0.223697
8  {'learning_rate': 0.019}         0.794313        0.223847


In [30]:
# SVR - USABLE

grid(SVR()).grid_get(X_scaled,y_log,{'C':[2.5,3,3.5],'kernel':["linear","poly","rbf"],'degree':[2,3]})

{'C': 2.5, 'degree': 2, 'kernel': 'rbf'} 0.8449253700650498
                                         params  mean_test_score  \
0   {'C': 2.5, 'degree': 2, 'kernel': 'linear'}         0.942174   
1     {'C': 2.5, 'degree': 2, 'kernel': 'poly'}         0.989409   
2      {'C': 2.5, 'degree': 2, 'kernel': 'rbf'}         0.844925   
3   {'C': 2.5, 'degree': 3, 'kernel': 'linear'}         0.942174   
4     {'C': 2.5, 'degree': 3, 'kernel': 'poly'}         1.001107   
5      {'C': 2.5, 'degree': 3, 'kernel': 'rbf'}         0.844925   
6     {'C': 3, 'degree': 2, 'kernel': 'linear'}         0.942160   
7       {'C': 3, 'degree': 2, 'kernel': 'poly'}         0.994583   
8        {'C': 3, 'degree': 2, 'kernel': 'rbf'}         0.845127   
9     {'C': 3, 'degree': 3, 'kernel': 'linear'}         0.942160   
10      {'C': 3, 'degree': 3, 'kernel': 'poly'}         1.020591   
11       {'C': 3, 'degree': 3, 'kernel': 'rbf'}         0.845127   
12  {'C': 3.5, 'degree': 2, 'kernel': 'linear'}         

In [105]:
# Ela

grid(ElasticNet()).grid_get(X_scaled,y_log,{'alpha':[0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.06]})

{'alpha': 0.062} 0.8527800921909167
             params  mean_test_score  std_test_score
0  {'alpha': 0.061}         0.852787        0.225596
1  {'alpha': 0.062}         0.852780        0.225825
2  {'alpha': 0.063}         0.852805        0.226078
3  {'alpha': 0.064}         0.852836        0.226331
4  {'alpha': 0.065}         0.852918        0.226717
5  {'alpha': 0.066}         0.853007        0.227124
6  {'alpha': 0.067}         0.853125        0.227497
7  {'alpha': 0.068}         0.853265        0.227898
8  {'alpha': 0.069}         0.853414        0.228348
9   {'alpha': 0.06}         0.852811        0.225372


In [31]:
# Bay - USABLE

grid(BayesianRidge()).grid_get(X_scaled,y_log,{'alpha_1':[1,5,10,20]})

{'alpha_1': 1} 0.8345174165603853
            params  mean_test_score  std_test_score
0   {'alpha_1': 1}         0.834517        0.208518
1   {'alpha_1': 5}         0.834889        0.208300
2  {'alpha_1': 10}         0.835363        0.208060
3  {'alpha_1': 20}         0.836329        0.207681


In [32]:
# Extra - USABLE

grid(ExtraTreesRegressor()).grid_get(X_scaled,y_log,{'max_features':["auto","sqrt","log2"],'bootstrap':[True]})

{'bootstrap': True, 'max_features': 'sqrt'} 0.8302189182442999
                                        params  mean_test_score  \
0  {'bootstrap': True, 'max_features': 'auto'}         0.833737   
1  {'bootstrap': True, 'max_features': 'sqrt'}         0.830219   
2  {'bootstrap': True, 'max_features': 'log2'}         0.851064   

   std_test_score  
0        0.184549  
1        0.245902  
2        0.270301  


In [54]:
# Ker

grid(KernelRidge()).grid_get(X_scaled,y_log,{'alpha':[0.1,0.2,0.3,0.4], 'kernel':["linear","polynomial"], 'degree':[2,3,4],'coef0':[0.1,0.5,0.8,0.9]})

{'alpha': 0.1, 'coef0': 0.9, 'degree': 2, 'kernel': 'polynomial'} 0.8341496409589338
                                               params  mean_test_score  \
0   {'alpha': 0.1, 'coef0': 0.1, 'degree': 2, 'ker...         0.869065   
1   {'alpha': 0.1, 'coef0': 0.1, 'degree': 2, 'ker...         0.853602   
2   {'alpha': 0.1, 'coef0': 0.1, 'degree': 3, 'ker...         0.869065   
3   {'alpha': 0.1, 'coef0': 0.1, 'degree': 3, 'ker...         0.874624   
4   {'alpha': 0.1, 'coef0': 0.1, 'degree': 4, 'ker...         0.869065   
..                                                ...              ...   
91  {'alpha': 0.4, 'coef0': 0.9, 'degree': 2, 'ker...         0.843385   
92  {'alpha': 0.4, 'coef0': 0.9, 'degree': 3, 'ker...         0.849509   
93  {'alpha': 0.4, 'coef0': 0.9, 'degree': 3, 'ker...         0.839915   
94  {'alpha': 0.4, 'coef0': 0.9, 'degree': 4, 'ker...         0.849509   
95  {'alpha': 0.4, 'coef0': 0.9, 'degree': 4, 'ker...         0.839642   

    std_test_score  
0    

In [33]:
# Xgb - USABLE 

grid(XGBRegressor()).grid_get(X_scaled,y_log,{'booster':["gbtree", "dart"],'tree_method':["auto", "exact", "approx", "hist"],'verbosity':[0]})

{'booster': 'dart', 'tree_method': 'auto', 'verbosity': 0} 0.8451390391112796
                                              params  mean_test_score  \
0  {'booster': 'gbtree', 'tree_method': 'auto', '...         0.845139   
1  {'booster': 'gbtree', 'tree_method': 'exact', ...         0.845139   
2  {'booster': 'gbtree', 'tree_method': 'approx',...         0.869342   
3  {'booster': 'gbtree', 'tree_method': 'hist', '...         0.851393   
4  {'booster': 'dart', 'tree_method': 'auto', 've...         0.845139   
5  {'booster': 'dart', 'tree_method': 'exact', 'v...         0.845139   
6  {'booster': 'dart', 'tree_method': 'approx', '...         0.869342   
7  {'booster': 'dart', 'tree_method': 'hist', 've...         0.851393   

   std_test_score  
0        0.205273  
1        0.205273  
2        0.200882  
3        0.188872  
4        0.205273  
5        0.205273  
6        0.200882  
7        0.188871  


In [35]:
# Ada - USABLE

grid(AdaBoostRegressor()).grid_get(X_scaled,y_log,{'learning_rate':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1],'loss':["linear", "square", "exponential"]})

{'learning_rate': 0.06, 'loss': 'square'} 0.7888133738133996
                                            params  mean_test_score  \
0        {'learning_rate': 0.01, 'loss': 'linear'}         0.799772   
1        {'learning_rate': 0.01, 'loss': 'square'}         0.805502   
2   {'learning_rate': 0.01, 'loss': 'exponential'}         0.809137   
3        {'learning_rate': 0.02, 'loss': 'linear'}         0.805829   
4        {'learning_rate': 0.02, 'loss': 'square'}         0.798567   
5   {'learning_rate': 0.02, 'loss': 'exponential'}         0.807075   
6        {'learning_rate': 0.03, 'loss': 'linear'}         0.805988   
7        {'learning_rate': 0.03, 'loss': 'square'}         0.798397   
8   {'learning_rate': 0.03, 'loss': 'exponential'}         0.808865   
9        {'learning_rate': 0.04, 'loss': 'linear'}         0.804694   
10       {'learning_rate': 0.04, 'loss': 'square'}         0.799177   
11  {'learning_rate': 0.04, 'loss': 'exponential'}         0.794366   
12       {'learn

# Ensemble Methods

In [36]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [37]:
# sorted by performance(good -> bad)
ada = AdaBoostRegressor(learning_rate = 0.06, loss = 'square')
rf = RandomForestRegressor(max_features = 'log2')
bay = BayesianRidge()
ridge = Ridge(alpha = 161)
extra = ExtraTreesRegressor(bootstrap = True, max_features = 'sqrt')
xgb = XGBRegressor(verbosity = 0)


In [38]:
# assign weights based on their gridsearch score

w1 = 0.25
w2 = 0.2
w3 = 0.2
w4 = 0.15
w5 = 0.15
w6 = 0.05

In [39]:
weight_avg = AverageWeight(mod = [ada,rf,bay,ridge,extra,xgb],weight=[w1,w2,w3,w4,w5,w6])

In [40]:
rmse_cv(weight_avg,X_scaled,y_log),  rmse_cv(weight_avg,X_scaled,y_log).mean()

(array([0.72458317, 0.6643149 , 0.93376797, 0.98046438, 0.64479545]),
 0.7853494822211787)

# Stacking

In [41]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [42]:
# must do imputer first, otherwise stacking won't work, and i don't know why.
a = Imputer().fit_transform(X_scaled)
b = Imputer().fit_transform(y_log.values.reshape(-1,1)).ravel()

C:\Users\Octavia\.conda\envs\MSBD5001\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Octavia\.conda\envs\MSBD5001\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [43]:
stack_model = stacking(mod = [ada,rf,bay,ridge,extra,xgb], meta_model = ada)

In [44]:
print(rmse_cv(stack_model,a,b))
print(rmse_cv(stack_model,a,b).mean())

[0.83692197 0.7250408  0.88650238 1.04488465 0.6245564 ]
0.794184899125049


### Extract the features generated from stacking, and combine them with original features.

In [45]:
X_train_stack, X_test_stack = stack_model.get_oof(a,b,test_X_scaled)

In [46]:
X_train_stack.shape, a.shape

((340, 6), (340, 26))

In [47]:
X_train_add = np.hstack((a,X_train_stack))

In [48]:
X_test_add = np.hstack((test_X_scaled,X_test_stack))

In [49]:
X_train_add.shape, X_test_add.shape

((340, 32), (90, 32))

In [50]:
print(rmse_cv(stack_model,X_train_add,b))
print(rmse_cv(stack_model,X_train_add,b).mean())

[0.77146262 0.6710715  0.93158308 0.98075866 0.75605872]
0.8160398248684368


# Submission

In [52]:
# Final Model

stack_model = stacking(mod = [ada,rf,bay,ridge,extra,xgb], meta_model = ada)

In [53]:
stack_model.fit(a,b)

stacking(meta_model=AdaBoostRegressor(base_estimator=None, learning_rate=0.06,
                                      loss='square', n_estimators=50,
                                      random_state=None),
         mod=[AdaBoostRegressor(base_estimator=None, learning_rate=0.06,
                                loss='square', n_estimators=50,
                                random_state=None),
              RandomForestRegressor(bootstrap=True, criterion='mse',
                                    max_depth=None, max_features='log2',
                                    max_leaf_nodes=None,
                                    min_i...
              XGBRegressor(base_score=0.5, booster='gbtree',
                           colsample_bylevel=1, colsample_bynode=1,
                           colsample_bytree=1, gamma=0, importance_type='gain',
                           learning_rate=0.1, max_delta_step=0, max_depth=3,
                           min_child_weight=1, missing=None, n_estimators=

In [54]:
pred = np.exp(stack_model.predict(test_X_scaled) + 1)

In [55]:
result=pd.DataFrame({'id':range(len(df_test2)), 'playtime_forever':pred})


In [56]:
result.to_csv("submission_1119_Standard.csv",index=False)